In [ ]:
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select  # Esta é a linha que você precisa adicionar
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import Select
# from selenium import webdriver

verify = False

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver = webdriver.Chrome()

verify = False

# Caminho do seu arquivo Excel
file_path = r"F:\Logistica_Florestal\EQUIPE\Sócrates Luis dos Santos\newPed\baseSgf.xlsx"

df = pd.read_excel(file_path)

filtered_df = df[df['Fornecedor'] == 'FM MODEL TRANSPORTES LTDA']
placas = filtered_df['Equip. Cavalo'].tolist()
# Filtrando por Tipo Conjunto ser TRITREM ou BITREM
tipo_conjunto_df = filtered_df[filtered_df['Tipo Conjunto'].isin(['TRITREM', 'BITREM'])]
if placas:
    primeira_placa = placas[0]  # Extrai a primeira placa
else:
    primeira_placa = "SEM_PLACA"  # Define um valor padrão caso não haja placas
    
fazendas = filtered_df['Projeto'].tolist()
if fazendas:
    primeira_fazenda = fazendas[0]  # Extrai a primeira fazenda
else:
    primeira_fazenda = "SEM_FAZENDA"  # Define um valor padrão caso não haja fazendas

# Formata o nome da fazenda para o padrão aceito pelo site
nome_fazenda_formatado = "FAZ " + primeira_fazenda + " - IDA"

webdriver_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=webdriver_service)
driver.get('https://app.viafacil.com.br/vpnew/loginEmbarcadorOut.do')
time.sleep(2)
cnpj_element = driver.find_element(By.XPATH, '//*[@id="embarcador_cnpj"]')
cnpj_element.send_keys('53943098000187')
usuario_element = driver.find_element(By.XPATH, '//*[@id="usuario_login"]')
usuario_element.send_keys('SLUIS')
senha_element = driver.find_element(By.XPATH, '//*[@id="embarcador_senha"]')
senha_element.send_keys('Br@cell123')
botao_ok = driver.find_element(By.XPATH, '//*[@id="btnLogin"]/img')
botao_ok.click()
time.sleep(5)
comprar_por_placa = driver.find_element(By.XPATH, '/html/body/table[2]/tbody/tr/th[1]/table[14]/tbody/tr/th[2]/a')
comprar_por_placa.click()
placa_element = driver.find_element(By.NAME, "veiculo")
placa_element.clear()
placa_element.send_keys(primeira_placa)  # Usa a primeira placa extraída
pesquisar_botao = driver.find_element(By.XPATH, '/html/body/form/table[2]/tbody/tr[3]/td/a/img')
pesquisar_botao.click()
# Parte de seleção da fazenda no dropdown com Selenium
select_element = WebDriverWait(driver, 20).until(
    EC.visibility_of_element_located((By.XPATH, '//*[@id="rota"]'))
)
select = Select(select_element)
select.select_by_visible_text(nome_fazenda_formatado)  # Seleciona a fazenda pelo texto visível formatado


# Calcula as datas
data_atual = datetime.now()
data_final = data_atual + timedelta(days=10)

# Formata as datas
data_atual_formatada = data_atual.strftime('%d/%m/%Y')
data_final_formatada = data_final.strftime('%d/%m/%Y')

# Preenche a data de início de vigência
campo_data_inicio = driver.find_element(By.XPATH, '/html/body/form/table[8]/tbody/tr/th/select[1]')
Select(campo_data_inicio).select_by_visible_text(data_atual_formatada)

# Preenche a data de término de vigência
campo_data_fim = driver.find_element(By.XPATH, '/html/body/form/table[8]/tbody/tr/th/select[2]')
Select(campo_data_fim).select_by_visible_text(data_final_formatada)
time.sleep(10)

# Localiza o botão pelo seu XPath e clica nele
botao_altera_eixo = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="divAlteraEixo"]'))
)
botao_altera_eixo.click()


# Localizar todos os elementos que correspondem à classe 'respostas_compra'
elementos_lista = driver.find_elements(By.CSS_SELECTOR, "tr.respostas_compra")
nome_desejado = "5 EIXOS ROD DUPLA"  # Defina o nome desejado aqui

# Inicializar 'indice_desejado' com None para verificar se encontramos o elemento
indice_desejado = None

# Percorrer todos os elementos da lista para encontrar o desejado
for indice, elemento in enumerate(elementos_lista):
    if nome_desejado in elemento.get_attribute('innerHTML'):
        indice_desejado = indice
        break

# Se um índice desejado foi encontrado, clique no elemento correspondente
if indice_desejado is not None:
    elementos_lista[indice_desejado].click()
    time.sleep(2)  # Aguarda 2 segundos para garantir que a página seja carregada
else:
    print("Nome desejado não encontrado.")

# Localizar o botão "Adicionar Viagem" e clicar nele
botao_adicionar_viagem = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "img[id='adicionarViagem']")))
botao_adicionar_viagem.click()
time.sleep(3)  # Espera por 3 segundos para garantir que a página seja carregada


In [ ]:
import pandas as pd

file_path = r"F:\Logistica_Florestal\EQUIPE\Sócrates Luis dos Santos\newPed\baseSgf.xlsx"
# Leitura do arquivo Excel
df = pd.read_excel(file_path)

# Lista para armazenar os resultados
resultados = []

# Obtendo os 20 primeiros fornecedores únicos
fornecedores_unicos = df['Fornecedor'].unique()[:50]

# Iterando sobre cada fornecedor único
for fornecedor in fornecedores_unicos:
    # Filtrando o DataFrame pelo fornecedor atual
    fornecedor_df = df[df['Fornecedor'] == fornecedor]
    
    # Filtrando por Tipo Conjunto ser TRITREM ou BITREM
    tipo_conjunto_df = fornecedor_df[fornecedor_df['Tipo Conjunto'].isin(['TRITREM', 'BITREM'])]
    
    # Verificando se existem registros após o filtro
    if not tipo_conjunto_df.empty:
        # Iterando sobre cada linha do DataFrame filtrado
        for _, row in tipo_conjunto_df.iterrows():
            # Criando um dicionário com as informações da linha atual
            info = {
                "Fornecedor": fornecedor,
                "Placa": row['Equip. Cavalo'],
                "Fazenda": row['Projeto'],
                "Tipo Conjunto": row['Tipo Conjunto']
            }
            # Adicionando o dicionário à lista de resultados
            resultados.append(info)

# Imprimindo os resultados
for resultado in resultados:
    print(resultado)


In [4]:
import pyautogui
import time

# Aguarda 5 segundos para você mover o mouse para a posição desejada
print("Mova o mouse para a posição desejada e aguarde 5 segundos")
time.sleep(4)

# Obtém e exibe as coordenadas atuais do mouse
posicao_atual = pyautogui.position()
print(f"Posição atual do mouse: {posicao_atual}")


Mova o mouse para a posição desejada e aguarde 5 segundos
Posição atual do mouse: Point(x=440, y=264)


In [ ]:
import pyautogui
import time

# As coordenadas onde você quer clicar
x, y = 983, 275

# Movendo o mouse para as coordenadas especificadas
pyautogui.moveTo(x, y, duration=1)  # Moverá o mouse para X=463, Y=1059 em 1 segundo

# Clicando na posição atual do mouse
pyautogui.click()

pyautogui.moveTo(x=1513, y=124, duration=1)  
pyautogui.click()


# Mover o cursor para a posição especificada
# Pedido
pyautogui.moveTo(x=1152, y=366, duration=1)
pyautogui.click()
pyautogui.hotkey('ctrl', 'c')  # Use 'command' em vez de 'ctrl' se estiver no Mac

pyautogui.moveTo(x=85, y=398, duration=1)
pyautogui.click()

pyautogui.moveTo(x=751, y=253, duration=1)
pyautogui.click()
pyautogui.hotkey('ctrl', 'v')

# remessa
pyautogui.moveTo(x=1152, y=386)
pyautogui.click()
pyautogui.hotkey('ctrl', 'c')

pyautogui.moveTo(x=85, y=398, duration=1)
pyautogui.click()

pyautogui.moveTo(x=847, y=251)
pyautogui.click()
pyautogui.hotkey('ctrl', 'v')

pyautogui.moveTo(x=1905, y=180)
pyautogui.click()
pyautogui.click()

#Voltando para o indice SAP
pyautogui.moveTo(x=993, y=296)
pyautogui.click()
pyautogui.click()


pyautogui.moveTo(x=1513, y=144, duration=1)  
pyautogui.click()

pyautogui.moveTo(x=1152, y=366, duration=1)
pyautogui.click()
pyautogui.hotkey('ctrl', 'c') 

pyautogui.moveTo(x=85, y=398, duration=1)
pyautogui.click()

pyautogui.moveTo(x=747, y=273, duration=1)
pyautogui.click()
pyautogui.hotkey('ctrl', 'v')







# Clicar para ver o número da nota
#pyautogui.moveTo(x=1109, y=478, duration=1)
#pyautogui.click()
#time.sleep(5)

# Copia o numero da nota
#pyautogui.moveTo(x=1623, y=241)
#pyautogui.click()
#pyautogui.hotkey('ctrl', 'c')

# Cola o número da nota
#pyautogui.moveTo(x=425, y=481, duration=2)
#pyautogui.click()
#time.sleep(2)
#pyautogui.click()
#pyautogui.hotkey('ctrl', 'v')



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from datetime import datetime, timedelta
import pandas as pd
import time
import pyautogui
import csv
from datetime import datetime



# Configurações para ignorar erros de certificado SSL
# options = webdriver.ChromeOptions()
# options.add_argument('--ignore-ssl-errors=yes')
# options.add_argument('--ignore-certificate-errors')

# Inicializa o WebDriver do Chrome com as opções definidas
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# driver = webdriver.Chrome(options=options)




# Parte 1: Análise de Dados do Excel
file_path =r"F:\Logistica_Florestal\EQUIPE\Sócrates Luis dos Santos\newPed\baseSgf.xlsx"  
df = pd.read_excel(file_path)
fornecedores = [
    'FM MODEL TRANSPORTES LTDA',
    'M.T. LOURENÇO TRANSPORTES EIRELI - ME',
    'ALEXANDRE APARECIDO CRUZ FRÓES EPP'
]

# Filtrar o DataFrame para incluir apenas as linhas com os fornecedores desejados
filtered_df = df[df['Fornecedor'].isin(fornecedores)]

# Iniciar o WebDriver
# webdriver_service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=webdriver_service)
driver = webdriver.Chrome()


    # Faça login uma vez antes de iniciar o loop



# Defina as placas dos caminhões específicos
placas_cinco_eixos_vazios = [
    'GAN8H82', 'GAQ2C35', 'GAW3J94', 'GAX8J03', 'GBD8F51',
    'GBT6J97', 'GBV8C54', 'GCI0I94', 'GCP2G73', 'GCQ8I34',
    'GCR4H22', 'GDR0J83', 'GDW4J22', 'GDW8E56', 'GDW8E68',
    'GDZ0E17', 'GEH7E88', 'GEV3J57', 'GFC2B18', 'GFD0J27',
    'GFI4G06', 'GFP6H82', 'GGO2I46', 'GGQ0E23', 'GGS6G25',
    'GGU6B12', 'GHA2A44', 'GHB1B84', 'GHL8I11', 'GHM3H15',
    'GHO0H65', 'GHS1H44', 'GHY0D77', 'GIE5D75', 'GII4G01',
    'GJH1I51', 'GJL7C45', 'GKD1A67', 'GGY0G27', 'GAD2C35',
    'GDK1A67', 'GFL5E52', 'GCIOI94', 'GDROJ83', 'GDZOE17',
    'GEP2F46', 'GEP4B98', 'GFC3B18', 'GFDOJ27', 'GGQ1C73',
    'GGQO23', 'GHL3A66', 'GIS9A85', 'GJL7S45'
    # Adicione mais placas conforme necessário
]

placas_seis_eixos_vazios = [
    'BRY4I91', 'DFM9D33', 'EEA7E41', 'EWW2B51', 'EXR3595',
    'EZR3J39', 'EZY2J75', 'FNI6A13', 'FVF0F64', 'FVP6B93',
    'GAS3995', 'GBZ4I07', 'GDK5732', 'GDR2A57', 'GEE8824',
    'GFM9506', 'GJS0D14', 'GJS2C26', 'GJU5A83', 'DRD0722',
    'GHK8A18', 'ELE1F95', 'GGL8287', 'EXV1996', 'GDN4200',
    'EWW2151', 'DZM5688', 'GJS0314', 'EZY2975', 'FMN5E49',
    'ELE8892', 'GBB8359', 'EUB8668', 'CUG3553', 'GHS4G74',
    'FXE4893', 'GDW7936', 'GFD5916', 'GHC6247', 'GKG9197',
    'GHO3376', 'BRY4891'
    # Adicione mais placas conforme necessário
]

# Iterar sobre cada linha do DataFrame filtrado
for index, row in filtered_df.iterrows():
    fornecedor = row['Fornecedor']
    primeira_placa = row['Equip. Cavalo'] if pd.notna(row['Equip. Cavalo']) else "SEM_PLACA"
    primeira_fazenda = row['Projeto'] if pd.notna(row['Projeto']) else "SEM_FAZENDA"
    fazendas = filtered_df['Projeto'].tolist()
    if fazendas:
        primeira_fazenda = fazendas[0]  # Extrai a primeira fazenda
    else:
        primeira_fazenda = "SEM_FAZENDA"  # Define um valor padrão caso não haja fazendas

    # Formata o nome da fazenda para o padrão aceito pelo site
    nome_fazenda_formatado = "FAZ " + primeira_fazenda + " - IDA"
    nome_fazenda_volta_formatado = "FAZ " + primeira_fazenda + " - VOLTA"

    # Verifica se o fornecedor é 'FM MODEL TRANSPORTES LTDA'
    if fornecedor == 'FM MODEL TRANSPORTES LTDA':
        if primeira_placa in placas_cinco_eixos_vazios:
            # Lógica específica para 5 eixos da FM MODEL TRANSPORTES LTDA
            primeiro_tipo_conjunto = 'BITREM'
            nome_desejado = "5 EIXOS ROD DUPLA"
            xpath_desejado = "//*[@id='alteraEixo']/table/tbody/tr[6]/td"
            nome_desejado_volta = "9 EIXOS ROD DUPLA"
            xpath_desejado_volta = "//*[@id='alteraEixo']/table/tbody/tr[10]/td"
            # Lógica para 6 eixos FM MODEL TRANSPORTES LTDA
        elif primeira_placa in placas_seis_eixos_vazios:
            primeiro_tipo_conjunto = 'TRITREM'
            nome_desejado = "6 EIXOS ROD DUPLA"
            xpath_desejado = "//*[@id='alteraEixo']/table/tbody/tr[7]/td"
            nome_desejado_volta = "9 EIXOS ROD DUPLA"
            xpath_desejado_volta = "//*[@id='alteraEixo']/table/tbody/tr[10]/td"
            
        else:
            # Lógica para outros estados ou quando a placa não está listada
            primeiro_tipo_conjunto = row['Tipo Conjunto'] if pd.notna(row['Tipo Conjunto']) else "SEM_TIPO"
    
    # Lógica para outros fornecedores
    elif fornecedor == 'M.T. LOURENÇO TRANSPORTES EIRELI - ME' or fornecedor == 'ALEXANDRE APARECIDO CRUZ FRÓES EPP':
        
        # Defina o nome desejado com base no tipo de conjunto
        if primeiro_tipo_conjunto == "TRITREM":
            nome_desejado = "6 EIXOS ROD DUPLA"
            xpath_desejado = "//*[@id='alteraEixo']/table/tbody/tr[7]/td"
            nome_desejado_volta = "9 EIXOS ROD DUPLA"
            xpath_desejado_volta = "//*[@id='alteraEixo']/table/tbody/tr[10]/td"
        elif primeiro_tipo_conjunto == "BITREM":
            nome_desejado = "5 EIXOS ROD DUPLA"
            xpath_desejado = "//*[@id='alteraEixo']/table/tbody/tr[6]/td"
            nome_desejado_volta = "7 EIXOS ROD DUPLA"
            xpath_desejado_volta = "//*[@id='alteraEixo']/table/tbody/tr[8]/td"
       
    driver.get('https://app.viafacil.com.br/vpnew/loginEmbarcadorOut.do')
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="embarcador_cnpj"]').send_keys('53943098000187')
    driver.find_element(By.XPATH, '//*[@id="usuario_login"]').send_keys('SLUIS')
    driver.find_element(By.XPATH, '//*[@id="embarcador_senha"]').send_keys('Br@cell123')
    driver.find_element(By.XPATH, '//*[@id="btnLogin"]/img').click()
    time.sleep(5)

    comprar_por_placa = driver.find_element(By.XPATH, '/html/body/table[2]/tbody/tr/th[1]/table[14]/tbody/tr/th[2]/a')
    comprar_por_placa.click()
    placa_element = driver.find_element(By.NAME, "veiculo")
    placa_element.clear()
    placa_element.send_keys(primeira_placa)  # Usa a primeira placa extraída
    pesquisar_botao = driver.find_element(By.XPATH, '/html/body/form/table[2]/tbody/tr[3]/td/a/img')
    pesquisar_botao.click()
    time.sleep(6) 

    try:  
        # Parte de seleção da fazenda no dropdown com Selenium
        select_element = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="rota"]'))
        )
        time
        select = Select(select_element)
        select.select_by_visible_text(nome_fazenda_formatado)  # Seleciona a fazenda pelo texto visível formatado
    except (NoSuchElementException, TimeoutException) as e:
        with open('erro_log.txt', 'a') as file:
            file.write(f"Falha na placa {primeira_placa} com fazenda {nome_fazenda_formatado}: {e}\n")
        print(f"Fazenda {nome_fazenda_formatado} não encontrada para a placa {primeira_placa}. Passando para a próxima.")
        continue


    # Calcula as datas
    data_atual = datetime.now()
    data_final = data_atual + timedelta(days=10)

    # Formata as datas
    data_atual_formatada = data_atual.strftime('%d/%m/%Y')
    data_final_formatada = data_final.strftime('%d/%m/%Y')

    # Preenche a data de início de vigência
    campo_data_inicio = driver.find_element(By.XPATH, '/html/body/form/table[8]/tbody/tr/th/select[1]')
    Select(campo_data_inicio).select_by_visible_text(data_atual_formatada)

    # Preenche a data de término de vigência
    campo_data_fim = driver.find_element(By.XPATH, '/html/body/form/table[8]/tbody/tr/th/select[2]')
    Select(campo_data_fim).select_by_visible_text(data_final_formatada)

    # Localiza o botão pelo seu XPath e clica nele
    botao_altera_eixo = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="divAlteraEixo"]'))
    )
    botao_altera_eixo.click()

    # mamipulando os eixos
    botao_eixo = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="alteraEixo"]/table/tbody/tr[10]/td')))
    botao_eixo.click()

    # Localiza o botão pelo seu XPath e clica nele
    botao_altera_eixo = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="divAlteraEixo"]'))
    )
    botao_altera_eixo.click()


    # Aguarde até que o elemento seja clicável e então clique nele
    elemento_desejado = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, xpath_desejado)))
    elemento_desejado.click()

    # Aguarde um pouco para garantir que a ação foi realizada
    time.sleep(2)  # Ajuste o tempo conforme necessário




        # Localiza e clica no botão 'Adicionar Viagem'
    botao_adicionar_viagem = WebDriverWait(driver, 6).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="adicionarViagem"]')))
    botao_adicionar_viagem.click()
    time.sleep(2)







    # Após adicionar a viagem de ida, iniciar a configuração da viagem de volta

    # Passo 1: Inserir a placa novamente e clicar em pesquisar
    placa_element = WebDriverWait(driver, 3).until(
        EC.visibility_of_element_located((By.NAME, "veiculo")))
    placa_element.clear()
    placa_element.send_keys(primeira_placa)  # Usa a mesma primeira placa extraída
    pesquisar_botao = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/form/table[2]/tbody/tr[3]/td/a/img')))
    pesquisar_botao.click()

    # Espera um pouco para a página carregar depois da pesquisa
    time.sleep(2)  # Ajuste este tempo conforme necessário

    # Passo 2: Selecionar a fazenda de volta no dropdown (altere 'nome_fazenda_formatado' conforme necessário)
    nome_fazenda_volta_formatado = "FAZ " + primeira_fazenda + " - VOLTA"  # Modifique conforme a sua lógica de nomes
    select_element = WebDriverWait(driver, 5).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="rota"]')))
    select = Select(select_element)
    select.select_by_visible_text(nome_fazenda_volta_formatado)

    # Calcula as datas
    data_atual = datetime.now()
    data_final = data_atual + timedelta(days=10)

    # Formata as datas
    data_atual_formatada = data_atual.strftime('%d/%m/%Y')
    data_final_formatada = data_final.strftime('%d/%m/%Y')

    # Preenche a data de início de vigência
    campo_data_inicio = driver.find_element(By.XPATH, '/html/body/form/table[8]/tbody/tr/th/select[1]')
    Select(campo_data_inicio).select_by_visible_text(data_atual_formatada)

    # Preenche a data de término de vigência
    campo_data_fim = driver.find_element(By.XPATH, '/html/body/form/table[8]/tbody/tr/th/select[2]')
    Select(campo_data_fim).select_by_visible_text(data_final_formatada)

    # Localiza o botão pelo seu XPath e clica nele
    botao_altera_eixo = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="divAlteraEixo"]'))
    )
    botao_altera_eixo.click()

    # mamipulando os eixos
    botao_eixo = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="alteraEixo"]/table/tbody/tr[10]/td')))
    botao_eixo.click()

    # Localiza o botão pelo seu XPath e clica nele
    botao_altera_eixo = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="divAlteraEixo"]'))
    )
    botao_altera_eixo.click()

    # Aguarde até que o elemento seja clicável e então clique nele
    elemento_desejado = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, xpath_desejado_volta)))
    elemento_desejado.click()





    # Passo 5: Clicar no botão para adicionar a viagem de volta
    botao_adicionar_viagem_volta = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="adicionarViagem"]')))
    botao_adicionar_viagem_volta.click()
    time.sleep(2)

    # Lembre-se de adicionar esperas adequadas e tratamentos de exceções conforme necessário

    botao_concluir = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="imgConcluir"]'))
    )
    botao_concluir.click()
    time.sleep(15)

    botao_imprimir_ida = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="row_0"]/td[5]/a/img'))
    )
    botao_imprimir_ida.click()
    time.sleep(15)
    

    # Defina as coordenadas para o clique
    x = 450
    y = 460

    # Move o mouse para a posição especificada e realiza o clique
    pyautogui.click(x, y, duration=1)
    time.sleep(15)
    botao_imprimir_volta = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="row_1"]/td[5]/a/img'))
    )
    botao_imprimir_volta.click()
    time.sleep(15)
    x=450
    y=460
    pyautogui.click(x, y, duration=2)
    time.sleep(15)

   

    # Abre o arquivo CSV no modo de acréscimo
    with open('placas_processadas.csv', 'a', newline='') as file:
        writer = csv.writer(file)
    # Escreve as informações no arquivo CSV
        writer.writerow([primeira_placa, datetime.now()])


# Fechar o navegador após completar as iterações (remova esta linha se desejar manter o navegador aberto)
    driver.quit()
    # webdriver_service = Service(ChromeDriverManager().install())
    # driver = webdriver.Chrome(service=webdriver_service)
    driver.get('https://app.viafacil.com.br/vpnew/loginEmbarcadorOut.do')  # Substitua pela sua URL 